# Notebook Transaksi Mesin Ramayana Makmur Sentosa Playzone Tahun 2024 - 2025

## Data Wrangling

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px  # Kita pakai Plotly agar grafik interaktif (cocok buat Streamlit)
import re
import matplotlib.ticker as ticker
import plotly.graph_objects as go
from rapidfuzz import fuzz

Load Data

In [ ]:
# Konfigurasi tampilan pandas
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.2f}'.format

# LOAD DATA
# Pastikan file ini adalah hasil output dari script terakhir Anda
file_path = "REKAP_DATA_MESIN_FULL.xlsx"
df = pd.read_excel(file_path)

print(f"Data Loaded: {df.shape[0]} baris, {df.shape[1]} kolom")
df.head()

Data Understanding : 
* Center = Nama toko
* Kode Game = Kode mesin individual
* Game Title = Nama mesin
* Keterangan = Nama mesin + jumlah player
* Kategori game = Jenis mesin
* Jumlah diaktifkan = Frekuensi mesin dimainkan per bulan
* Kredit yang digunakan = Jumlah uang yang dikeluarkan pemain pada suatu mesin per bulan
* Bonus yang digunakan = 
* Main Gratis/Koin Masuk = 
* QRIS = 
* Total = 
* Tiket/Koin Keluar = 
* Mercy Ticket = 
* Payout = 
* Avg. Ticket/Coin Out Per Play/Coin In = 
* Bulan = Periode bulan transaksi mesin
* Tahun = Periode tahun transaksi mesin
* Asal_Folder = Asal folder lokal data discraping
* Nama_File_Asal = Asal file lokal data discraping

Assesing Data

In [ ]:
df.info()

Dataframe df memiliki 147930 - 153452 entry data. Terdapat . Terdapat 5.522 data yang hilang. Untuk kolom tahun dan bulan masih berbentuk integer dan object sehingga perlu preprocessing menjadi datetime agar dapat difilter nantinya.

*Pemeriksaan parameter statistik*

In [ ]:
df.describe()

* Tidak ada outlier yang signifikan
* Kolom QRIS dan Mercy Ticket kosong sama sekali sehingga bisa didrop
* kolom Main Gratis/Koin Masuk,  Tiket/Koin Keluar, Payout, dan Avg. Ticket/Coin Out Per Play/Coin In banyak value yang kosong sehingga bisa didrop.

## Data Cleaning & Preprocessing

Drop kolom yang seluruhnya kosong dan mayoritas kosong karena sedikit informasinya. Serta asal folder dan file karena kurang informarif untuk analisis.

In [ ]:
cols_to_drop = [
    'QRIS', 
    'Mercy Ticket', 
    'Main Gratis/Koin Masuk', 
    'Tiket/Koin Keluar', 
    'Payout', 
    'Avg. Ticket/Coin Out Per Play/Coin In',
    'Asal_Folder',
    'Nama_File_Asal'

]
df.drop(columns=cols_to_drop, axis=1, inplace=True, errors='ignore')
display(df.head())

Pemeriksaan duplikasi

In [ ]:
print("Jumlah data duplikat dari df:", df.duplicated().sum())

Ada data duplikat, maka perlu dihandle.

In [ ]:
df_clean = df.drop_duplicates()

Cek kembali duplikat.

In [ ]:
print("Jumlah data duplikat dari df setelah dihandle:", df_clean.duplicated().sum())

Lihat null di setiap kolom

In [ ]:
# Menghitung jumlah null di setiap kolom
print(df.isna().sum())

Lihat presentase missing values

In [ ]:
# Menampilkan jumlah dan persentase
missing_data = df.isna().sum()
total_rows = len(df)

percent_missing = (missing_data / total_rows) * 100

# Gabungkan jadi satu tabel agar mudah dibaca
missing_df = pd.DataFrame({
    'Total Missing': missing_data,
    'Persentase (%)': percent_missing
})

# Tampilkan hanya kolom yang ada missing values-nya (urut dari terbesar)
print(missing_df[missing_df['Total Missing'] > 0].sort_values('Persentase (%)', ascending=False))

Cek baris yang kosong

In [ ]:
# Menampilkan baris yang memiliki setidaknya satu kolom kosong
baris_kosong = df[df.isna().any(axis=1)]

display(baris_kosong.head())

Drop missing value

In [ ]:
# Kolom yang ingin di drop
df.dropna(subset=['Center', 'Kode Game', 'Game Title'], inplace=True)

# Reset index agar urut kembali
df.reset_index(drop=True, inplace=True)

Cek kembali statistik umum setelah cleaning data

In [ ]:
df_clean.info()

Terdapat pengurangan banyaknya data menjadi 147931

**Cleaning data waktu agar datetime**

Mapping Bulan dan Tahun ke Tanggal (datetime) agar mudah dikategorikan

In [ ]:
# Standardisasi Nama Bulan ---
# Pastikan nama bulan konsisten (Title Case: Januari, bukan januari)
# Menggunakan df_clean
df_clean['Bulan'] = df_clean['Bulan'].astype(str).str.title()

# Mapping Bulan Indonesia ke Angka (Untuk bikin tanggal)
map_bulan = {
    'Januari': 1, 'Februari': 2, 'Maret': 3, 'April': 4,
    'Mei': 5, 'Juni': 6, 'Juli': 7, 'Agustus': 8,
    'September': 9, 'Oktober': 10, 'November': 11, 'Desember': 12
}

df_clean['Bulan_Angka'] = df_clean['Bulan'].map(map_bulan)

# Membuat Kolom Tanggal (Date) ---
# Kita set tanggal 1 sebagai default karena data kita bulanan
# Menggunakan df_clean.Tahun dan df_clean.Bulan_Angka
df_clean['Tanggal'] = pd.to_datetime(dict(year=df_clean.Tahun, month=df_clean.Bulan_Angka, day=1))

# Filter Data Aneh (Optional) ---
# Contoh: Hapus jika Total_Sales negatif (jika perlu)
# df_clean = df_clean[df_clean['Total_Sales'] >= 0] 

print("Rentang Data:", df_clean['Tanggal'].min(), "sampai", df_clean['Tanggal'].max())
df_clean.info()

**Filter center yang aktif saja dan mapping standarisasi nama toko**

Map Toko Aktif

In [ ]:
# Key (Kiri) = Nama di Center
# Value (Kanan) = Nama Baru Standar
mapping_center = {
    # --- AREA R ---
    "Abepura (R-91)": "R091 Abepura",
    "Samarinda (R-94) RMS": "R094 Samarinda",
    "Panam (R-96)": "R096 Panam",
    "Kediri (R-98) RMS": "R098 Kediri",
    "Garut (R-99) RMS": "R099 Garut",
    "Sabar Subur": "R135 SS Tangerang",       
    "Cimone (R-26) IFS": "R026 Cimone",
    "Plaza Sukaramai Pekanbaru (R-56)": "R056 Pekanbaru",
    "Cirebon (R-100)": "R100 Cirebon",
    "Rajabasa Lampung": "R101 Rajabasa",
    "Kebayoran Lama (R-102) IFS": "R102 Kebayoran Lama",
    "Sorong (R-103)": "R103 Sorong",
    "Parung": "R105 Parung",
    "Cibadak (R-106)": "R106 Cibadak",
    "Pekalongan (R-110)": "R110 Pekalongan",
    "Yasmin Bogor": "R114 Bogor Yasmin",
    "Dinoyo": "R119 Dinoyo",
    "Tajur (R-121)": "R121 Tajur",
    "Cikupa": "R124 Cikupa",
    "Jatinegara": "R125 Jatinegara",
    "Timika (R-138)": "R138 Timika",
    "Dewi Sartika Bogor": "R015 Dewi Sartika",

    # --- AREA RB ---
    "Sukabumi (RB-05) RMS": "RB05 Sukabumi",
    "Cimahi (RB-12)": "RB12 Cimahi",
    "Gresik (RB-16) RMS": "RB16 Gresik",
    "Denpasar (RB-20) RMS": "RB20 Denpasar",
    "Sidoarjo (RB-27) RMS": "RB27 Sidoarjo",
    "Kudus (RB-28)": "RB28 Kudus",
    "Panbil Batam (RB-30) RMS": "RB30 Panbil",
    "Malang (RB-M) RMS": "RBM Malang",
    "Krian (RB-23)": "RB23 Krian",

    # --- AREA ZD ---
    "Ubud Bali": "ZD15 Bali Batu Bulan",
    "Wonosari": "ZD25 Wonosari",
    "Ponorogo (ZD-44)": "ZD44 Ponorogo",
    "Lotte Bintaro (ZD-46)": "ZD46 Lotte Bintaro",
    "Payakumbuh": "ZD39 Payakumbuh",
    "Mesra Samarinda (ZD-37)": "ZD37 Samarinda"
}

Filter dan Rename

In [ ]:
# Bersihkan spasi berlebih agar pencocokan akurat
df['Center'] = df['Center'].astype(str).str.strip()

# Langkah A: Filter (Hanya ambil data yang Center-nya ada di daftar mapping)
df_clean = df[df['Center'].isin(mapping_center.keys())].copy()

# Langkah B: Rename (Ubah nama lama jadi nama baru)
df_clean['Center'] = df_clean['Center'].map(mapping_center)

# ================= 3. CEK HASIL =================
print(f"Data Awal: {len(df)} baris")
print(f"Data Setelah Filter Center Aktif: {len(df_clean)} baris")
print("\nDaftar Center Aktif (Standardized):")
print(df_clean['Center'].unique())

# Update ke dataframe utama
df = df_clean

Coba Mapping 'Game Title'

In [ ]:
import re

def normalize_game_title(text: str) -> str:
    if not isinstance(text, str):
        return text

    text = text.upper()
    text = re.sub(r"\s+", " ", text)      # multiple space → single
    text = text.strip()
    return text


In [ ]:
def pre_clean(text):
    text = normalize_game_title(text)
    text = re.sub(r"\s?#\d+|\s?\([A-Z]\)", "", text)
    text = re.sub(r"\b(SD|TWIN|SINGLE|DELUXE|DX|JR|MINI|BIG)\b", "", text)
    return text.strip()


In [ ]:
game_title_map = {
    # ================= AIR HOCKEY =================
    "JUMBO AIR HOCKEY 2000": "JUMBO AIR HOCKEY",
    "PAIR MATCH HOCKEY": "JUMBO AIR HOCKEY",

    # ================= RAINBOW =================
    "RAINBOW SPINNER": "RAINBOW SPINNER",
    "RAINBOW SPINER": "RAINBOW SPINNER",
    "RANBOW SPINNER": "RAINBOW SPINNER",
    "RAINBOW SPINER": "RAINBOW SPINNER",
    "RAINBOW": "RAINBOW",

    # ================= SPEED DRIVER =================
    "SPEED DRIVER": "SPEED DRIVER",
    "SPEED DRIVER 3 SD TWIN": "SPEED DRIVER",
    "CRAZY SPEED": "SPEED DRIVER",
    "CRAZY SPEED SD SINGLE": "SPEED DRIVER",
    "CRAZY SPEED TWIN": "SPEED DRIVER",

    # ================= SPEED RIDER =================
    "SPEED RIDER": "SPEED RIDER",
    "SPEED RIDER 2 TWIN": "SPEED RIDER",
    "SPEED RIDER 3": "SPEED RIDER",
    "SPEED RIDER 3D": "SPEED RIDER",

    # ================= SPEEDING UP =================
    "SPEEDING UP": "SPEEDING UP",

    # ================= MAXIMUM TUNE =================
    "MAXIMUM TUNE": "MAXIMUM TUNE",

    # ================= RACING TUNED =================
    "RACING TUNED": "RACING TUNED",
    "R TUNED": "RACING TUNED",
    "R-TUNED RACING": "RACING TUNED",

    # ================= KERETA =================
    "KERETA KENCANA": "CLASSIC WAGOON",
    "KERETA GOYANG": "CLASSIC WAGOON",
    "KERETA WAHANA": "CHOOCHOO TRAIN",

    # ================= ZAMPERLA =================
    "ZAMPERLA HORSE": "ZAMPERLA RIDE",
    "ZAMPERLA UNTA": "ZAMPERLA RIDE",

    # ================= POP IT WIN =================
    "POP IT WIN": "POP IT WIN",
    "POP IT & WIN": "POP IT WIN",
    "POP IT N WIN": "POP IT WIN",
    "POT IT WIN": "POP IT WIN",
    "PO IT WIN": "POP IT WIN",
}


In [ ]:
df["Game Title Clean"] = (
    df["Game Title"]
    .apply(normalize_game_title)
    .replace(game_title_map)
)


In [ ]:
df.groupby("Game Title Clean")["Game Title"].nunique() \
  .sort_values(ascending=False) \
  .head(20)


In [ ]:
conflict = (
    df.assign(
        GT_NORM=df["Game Title"].apply(normalize_game_title),
        GT_CLEAN=lambda x: x["GT_NORM"].replace(game_title_map)
    )
    .groupby("GT_CLEAN")["GT_NORM"]
    .nunique()
    .reset_index(name="jumlah_variasi")
    .query("jumlah_variasi > 1")
    .sort_values("jumlah_variasi", ascending=False)
)

conflict


In [ ]:
detail = (
    df.assign(
        GT_NORM=df["Game Title"].apply(normalize_game_title),
        GT_CLEAN=lambda x: x["GT_NORM"].replace(game_title_map)
    )
    .query("GT_CLEAN in @conflict.GT_CLEAN")
    [["Game Title", "GT_NORM", "GT_CLEAN"]]
    .sort_values("GT_CLEAN")
)

detail


In [ ]:
def master_clean(text):
    text = normalize_game_title(text)

    text = re.sub(r"\s?#\d+|\s?\([A-Z]\)", "", text)

    if re.search(r"POP\s*I?T?\s*(N|&)?\s*WIN", text):
        return "POP IT WIN"

    if "CRAZY SPEED" in text or "SPEED DRIVER" in text:
        return "SPEED DRIVER"

    if "AIR HOCKEY" in text or "PAIR MATCH HOCKEY" in text:
        return "JUMBO AIR HOCKEY"

    if text in ["KERETA GOYANG", "KERETA KENCANA", "CLASSIC WAGON", "CLASSIC WAGOON"]:
        return "CLASSIC WAGOON"

    if re.search(r"R.?AINBOW\s*SPIN", text):
        return "RAINBOW SPINNER"

    if re.search(r"\bR\s*-?\s*TUNED\b", text) or "RACING TUNED" in text:
        return "RACING TUNED"

    if text in ["CHOOCHOO TRAIN", "KERETA WAHANA"]:
        return "CHOOCHOO TRAIN"

    if text.startswith("ZAMPERLA"):
        return "ZAMPERLA RIDE"

    return text


In [ ]:
df["GT_FINAL"] = df["Game Title"].apply(master_clean)

df.groupby("GT_FINAL")["GT_FINAL"].nunique().sort_values(ascending=False)


Dapatkan unique value untuk GT_FINAL

In [ ]:
titles = sorted(df["GT_FINAL"].unique())

Fuzzy Similarity

In [ ]:
rows = []

for i, a in enumerate(titles):
    for b in titles[i+1:]:
        score = fuzz.token_sort_ratio(a, b) / 100
        if score >= 0.85:   # threshold audit
            rows.append({
                "GT_A": a,
                "GT_B": b,
                "similarity": round(score, 2)
            })

audit_df = pd.DataFrame(rows).sort_values("similarity", ascending=False)
audit_df


Ekspor Audit Excel

In [ ]:
'''
output_path = "audit_game_title_similarity.xlsx"

audit_df.to_excel(
    output_path,
    index=False,
    sheet_name="Audit Similarity"
)

print(f"Audit berhasil diekspor ke {output_path}")
'''

Baca Audit Merge Excel ke Dataframe

In [ ]:
audit_df = pd.read_excel("audit_merge_map.xlsx")

audit_merge_map = dict(
    zip(
        audit_df["GT_B"].str.upper().str.strip(),
        audit_df["GT_A"].str.upper().str.strip()
    )
)


APPLY ke GT_FINAL

In [ ]:
df["GT_FINAL"] = (
    df["GT_FINAL"]
    .str.upper()
    .str.strip()
    .replace(audit_merge_map)
)


Validasi WAJIB

In [ ]:
set(audit_merge_map.keys()) & set(df["GT_FINAL"].unique())


Semua GT_A harus ADA

In [ ]:
set(audit_merge_map.values()) - set(df["GT_FINAL"].unique())


In [ ]:
df.groupby("GT_FINAL")["Game Title"].nunique() \
  .sort_values(ascending=False) \
  .head(20)


DEFINISI NORMALIZER (FINAL)

In [ ]:
def normalize_game_title(x: str) -> str:
    if pd.isna(x):
        return x
    x = x.upper()
    x = re.sub(r"[^\w\s]", " ", x)
    x = re.sub(r"\s+", " ", x)
    return x.strip()


LOAD & BUILD AUDIT MERGE MAP (FINAL)

In [ ]:
audit_df = pd.read_excel("audit_merge_map.xlsx")

audit_merge_map = dict(
    zip(
        audit_df["GT_B"].apply(normalize_game_title),
        audit_df["GT_A"].apply(normalize_game_title)
    )
)


APPLY KE DATA (FINAL APPLY)

In [ ]:
df["GT_FINAL"] = df["GT_FINAL"].apply(normalize_game_title)
df["GT_FINAL"] = df["GT_FINAL"].replace(audit_merge_map)


VALIDASI WAJIB

In [ ]:
remaining = set(audit_merge_map.keys()) & set(df["GT_FINAL"].unique())
remaining


MANUAL MAPPING

In [ ]:
manual_fix_map = {
    "E TICKET": "E-TICKET",
    "MAXIMUM TUNE 3": "MAXIMUM TUNE",
    "MAXIMUM TUNE 3 DX": "MAXIMUM TUNE",
    "TROLEY": "TROLLEY"
}


Apply manual fix

In [ ]:
df["GT_FINAL"] = df["GT_FINAL"].replace(manual_fix_map)


VALIDASI FINAL (HARUS KOSONG)

In [ ]:
{
    'E TICKET',
    'MAXIMUM TUNE 3',
    'MAXIMUM TUNE 3 DX',
    'TROLEY'
} & set(df["GT_FINAL"].unique())


In [ ]:
remaining = set(final_merge_map.keys()) & set(df["GT_FINAL"].unique())
remaining


APAKAH STRING-NYA BENAR-BENAR IDENTIK?

In [ ]:
for x in remaining:
    print(repr(x))


EKSPOR DF HASIL NORMALISASI

In [ ]:
print("Kolom df:")
print(df.columns.tolist())

print("\nKolom df_clean:")
print(df_clean.columns.tolist())


In [ ]:
(df['GT_FINAL'] == df_clean['GT_FINAL']).value_counts()


In [ ]:
df_clean['GT_FINAL'].value_counts().sum()

BUAT TANGGAL

In [ ]:
# Pastikan Bulan sudah title-case
df_clean['Bulan'] = df_clean['Bulan'].astype(str).str.title()

map_bulan = {
    'Januari': 1, 'Februari': 2, 'Maret': 3, 'April': 4,
    'Mei': 5, 'Juni': 6, 'Juli': 7, 'Agustus': 8,
    'September': 9, 'Oktober': 10, 'November': 11, 'Desember': 12
}

df_clean['Bulan_Angka'] = df_clean['Bulan'].map(map_bulan)

# Buat kolom Tanggal (tanggal 1 tiap bulan)
df_clean['Tanggal'] = pd.to_datetime(
    dict(
        year=df_clean['Tahun'],
        month=df_clean['Bulan_Angka'],
        day=1
    )
)

# Validasi
print("Kolom sekarang:")
print(df_clean.columns.tolist())

print("Range tanggal:")
print(df_clean['Tanggal'].min(), "→", df_clean['Tanggal'].max())


LANGKAH FINAL YANG WAJIB DIKUNCI (BIAR STABIL)

In [ ]:
df_clean = df.copy()


EKSPOR DATA MESIN

In [ ]:
cols_export = [
    'Center',
    'GT_FINAL',
    'Kode Game',
    'Keterangan',
    'Kategori Game',
    'Jumlah Diaktifkan',
    'Kredit yg Digunakan',
    'Tanggal'
]

df_clean[cols_export].to_excel(
    "CLEAN_DATA_MESIN_FINAL.xlsx",
    index=False
)


EDA

In [ ]:
# Konfigurasi tampilan pandas
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.2f}'.format

# LOAD DATA
# Pastikan file ini adalah hasil output dari script terakhir Anda
file_path = "CLEAN_DATA_MESIN_FINAL.xlsx"
df_mesin = pd.read_excel(file_path)

print(f"Data Loaded: {df_mesin.shape[0]} baris, {df_mesin.shape[1]} kolom")
df_mesin.head()

In [ ]:
exclusions = [
    'KIDDIE LAND', 
    'KIDDIE LAND 1 JAM', 
    'KIDDIELAND MINI', 
    'KIDDIELAND SEPUASNYA',
    'KIDDIE ZONE 1 JAM'
]

# Pastikan string rapi
df_mesin['Kategori Game'] = df_mesin['Kategori Game'].astype(str).str.strip().str.upper()

# Filter exclude
df_mesin_excl = df_mesin[
    ~df_mesin['Kategori Game'].isin(exclusions)
].copy()

print("Sebelum exclude:", len(df_mesin))
print("Sesudah exclude :", len(df_mesin_excl))


In [ ]:


mesin_trend = (
    df_mesin_excl
    .groupby(pd.Grouper(key='Tanggal', freq='M'))
    [['Jumlah Diaktifkan', 'Kredit yg Digunakan']]
    .sum()
)

fig, ax1 = plt.subplots(figsize=(10,4))

# ===== Axis Kiri: Jumlah Diaktifkan =====
ax1.plot(
    mesin_trend.index,
    mesin_trend['Jumlah Diaktifkan'],
    marker='o',
    label='Jumlah Diaktifkan'
)
ax1.set_ylabel("Jumlah Diaktifkan")
ax1.yaxis.set_major_formatter(
    ticker.FuncFormatter(lambda x, _: f"{int(x):,}".replace(",", "."))
)

# ===== Axis Kanan: Kredit yg Digunakan (Rupiah) =====
ax2 = ax1.twinx()
ax2.plot(
    mesin_trend.index,
    mesin_trend['Kredit yg Digunakan'],
    marker='s',
    linestyle='--',
    label='Kredit Digunakan (Rp)'
)
ax2.set_ylabel("Kredit Digunakan (Rp)")
ax2.yaxis.set_major_formatter(
    ticker.FuncFormatter(lambda x, _: f"Rp {x:,.0f}".replace(",", "."))
)

# ===== Judul & Legend =====
fig.suptitle("Tren Aktivitas Mesin", fontsize=12)

lines_1, labels_1 = ax1.get_legend_handles_labels()
lines_2, labels_2 = ax2.get_legend_handles_labels()

ax1.legend(
    lines_1 + lines_2,
    labels_1 + labels_2,
    loc='upper left'
)

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

# Pastikan kolom tanggal datetime
df_mesin_excl['Tanggal'] = pd.to_datetime(df_mesin_excl['Tanggal'])

# Tambah kolom Tahun
df_mesin_excl['Tahun'] = df_mesin_excl['Tanggal'].dt.year

# Filter hanya 2024 & 2025
df_comp = df_mesin_excl[df_mesin_excl['Tahun'].isin([2024, 2025])]

# Agregasi omset per tahun
omset_tahun = (
    df_comp
    .groupby('Tahun')['Kredit yg Digunakan']
    .sum()
    .reset_index()
)


In [ ]:
fig, ax = plt.subplots(figsize=(6,4))

ax.bar(
    omset_tahun['Tahun'].astype(str),
    omset_tahun['Kredit yg Digunakan']
)

# Format Rupiah Indonesia di Y-axis
ax.yaxis.set_major_formatter(
    ticker.FuncFormatter(lambda x, _: f"Rp {x:,.0f}".replace(",", "."))
)

ax.set_title("Komparasi Omset Mesin 2024 vs 2025")
ax.set_xlabel("Tahun")
ax.set_ylabel("Total Omset Mesin")

# Label di atas bar
for idx, row in omset_tahun.iterrows():
    ax.text(
        idx,
        row['Kredit yg Digunakan'],
        f"Rp {row['Kredit yg Digunakan']:,.0f}".replace(",", "."),
        ha='center',
        va='bottom',
        fontsize=9
    )

plt.tight_layout()
plt.show()


Terjadi kesalahan di value, coba reset workflow.

# =========================================
# NOTEBOOK RESET – MESIN DASHBOARD
# Last reset: Jan 2026
# Semua cell di bawah harus dijalankan berurutan
# =========================================


In [ ]:
df = pd.read_excel("REKAP_DATA_MESIN_FULL.xlsx")

print(df.columns)


In [ ]:
df.info()
df.isna().sum()

tidak ada angka kredit yang “nggandain” lagi.

In [ ]:
df[['Jumlah Diaktifkan', 'Kredit yg Digunakan']].describe()


cek cepat logika

In [ ]:
print("Max Jumlah:", df['Jumlah Diaktifkan'].max())
print("Max Kredit:", df['Kredit yg Digunakan'].max())


Buat kolom tanggal datetime

In [ ]:
bulan_map = {
    'Januari': 1, 'Februari': 2, 'Maret': 3, 'April': 4,
    'Mei': 5, 'Juni': 6, 'Juli': 7, 'Agustus': 8,
    'September': 9, 'Oktober': 10, 'November': 11, 'Desember': 12
}

df['Bulan_num'] = df['Bulan'].map(bulan_map)

df['Tanggal'] = pd.to_datetime(
    dict(year=df['Tahun'].astype(int), month=df['Bulan_num'], day=1)
)


Standarisasi Game Title

In [ ]:
df['Game Title'] = (
    df['Game Title']
    .astype(str)
    .str.strip()
    .str.upper()
)


Normalizer

In [ ]:
def normalize_gt(s):
    if pd.isna(s):
        return s
    return (
        str(s)
        .upper()
        .strip()
        .replace('-', ' ')
        .replace('_', ' ')
    )


Aplikasi normalisasi

In [ ]:
df['GT_RAW_NORM'] = df['Game Title'].apply(normalize_gt)


LOAD MAPPING & APPLY (INTI PERBAIKAN)

In [ ]:
map_gt = pd.read_excel("mapping_gt.xlsx")

map_gt['GT_RAW_NORM'] = map_gt['GT_RAW'].apply(normalize_gt)

gt_dict = dict(
    zip(map_gt['GT_RAW_NORM'], map_gt['GT_FINAL'])
)

Apply mapping:

In [ ]:
df['GT_FINAL'] = df['GT_RAW_NORM'].map(gt_dict)
df['GT_FINAL'] = df['GT_FINAL'].fillna(df['GT_RAW_NORM'])


AUDIT YANG SEHARUSNYA

In [ ]:
audit_gt = (
    df.groupby('GT_FINAL')['GT_RAW_NORM']
    .nunique()
    .reset_index(name='jumlah_variasi')
    .sort_values('jumlah_variasi', ascending=False)
)


Lihat yang perlu merge lagi:

In [ ]:
audit_gt[audit_gt['jumlah_variasi'] > 1]


KLASIFIKASI 127 VARIASI

In [ ]:
audit_detail = (
    df[['GT_RAW_NORM', 'GT_FINAL']]
    .drop_duplicates()
    .sort_values('GT_FINAL')
)


Export untuk review:

In [ ]:
'''
audit_detail.to_excel("AUDIT_GT_VARIASI.xlsx", index=False)
'''

Kebanyakan karena belum filter toko aktif

In [ ]:
DICT_TOKO_NTA = {

    # ================== AREA R ==================
    "Ciputat (R-20)": "R020 Ciputat",
    "Bekasi (R-21)": "R021 Bekasi",
    "Pasar Minggu (R-22)": "R022 Pasar Minggu",
    "Tanjung Priok": "R029 Tanjung Priuk",
    "Tangerang (R-31) RMS": "R031 Tangerang",
    "Depok (R-34) RMS": "R034 Depok",
    "Cilegon (R-35) RMS": "R035 Cilegon",
    "Cibitung (R-37) RMS": "R037 Cibitung",
    "Klender (R-38) RMS": "R038 Klender 1",
    "Cengkareng (R-40)": "R040 Cengkareng",
    "Pematang Siantar (R-41) RMS": "R041 Siantar",
    "Jambi (R-45) RMS": "R045 Jambi",
    "Tanjung Karang Lampung": "R048 Lampung",
    "Ciledug (R-49) RMS": "R049 Ciledug",
    "Balikpapan (R-51)": "R051 Balikpapan",
    "Pangkal Pinang (R-52) RMS": "R052 Pangkal Pinang",
    "Pontianak (R-57)": "R057 Pontianak",
    "Medan (R-58) RMS": "R058 Medan 2",
    "Tarakan (R-60)": "R060 Tarakan",
    "Kupang (R-61) RMS": "R061 Kupang",
    "Banjarmasin": "R063 Banjarmasin",
    "Padang (R-66) RMS": "R066 Padang",
    "Mega Mal Karawang (R-67) RMS": "R067 Karawang",
    "Makasar 2 (R-70)": "R070 Makassar 2",
    "Bogor Trade Mall (R-71) RMS": "R071 BTM",
    "Serang (R-75) RMS": "R075 Serang",
    "Bukittinggi (R-77) RMS": "R077 Bukittinggi",
    "Makasar 3 (R-79) RMS": "R079 Makassar 3",
    "Dumai (R-80)": "R080 Dumai",
    "Bontang (R-81)": "R081 Bontang",
    "Banyuwangi": "R083 Banyuwangi",
    "Kerinci (R-86)": "R086 Kerinci",
    "Tebing Tinggi (R-89) RMS": "R089 Tebing Tinggi",
    "Abepura (R-91)": "R091 Abepura",
    "Samarinda (R-94) RMS": "R094 Samarinda",
    "Panam (R-96)": "R096 Panam",
    "Kediri (R-98) RMS": "R098 Kediri",
    "Garut (R-99) RMS": "R099 Garut",
    "Sabar Subur": "R135 SS Tangerang",

    # ================== AREA RB ==================
    "Sukabumi (RB-05) RMS": "RB05 Sukabumi",
    "Cimahi (RB-12)": "RB12 Cimahi",
    "Gresik (RB-16) RMS": "RB16 Gresik",
    "Denpasar (RB-20) RMS": "RB20 Denpasar",
    "Sidoarjo (RB-27) RMS": "RB27 Sidoarjo",
    "Kudus (RB-28)": "RB28 Kudus",
    "Panbil Batam (RB-30) RMS": "RB30 Panbil",
    "Malang (RB-M) RMS": "RBM Malang",
    "Krian (RB-23)": "RB23 Krian",

    # ================== AREA ZD ==================
    "Ubud Bali": "ZD15 Bali Batu Bulan",
    "Wonosari": "ZD25 Wonosari",
    "Ponorogo (ZD-44)": "ZD44 Ponorogo",
    "Lotte Bintaro (ZD-46)": "ZD46 Lotte Bintaro",
    "Payakumbuh": "ZD39 Payakumbuh",
    "Mesra Samarinda (ZD-37)": "ZD37 Samarinda",

    # ================== TAMBAHAN R ==================
    "Dewi Sartika Bogor": "R015 Dewi Sartika",
    "Cimone (R-26) IFS": "R026 Cimone",
    "Plaza Sukaramai Pekanbaru (R-56)": "R056 Pekanbaru",
    "Cirebon (R-100)": "R100 Cirebon",
    "Rajabasa Lampung": "R101 Rajabasa",
    "Kebayoran Lama (R-102) IFS": "R102 Kebayoran Lama",
    "Sorong (R-103)": "R103 Sorong",
    "Parung": "R105 Parung",
    "Cibadak (R-106)": "R106 Cibadak",
    "Pekalongan (R-110)": "R110 Pekalongan",
    "Yasmin Bogor": "R114 Bogor Yasmin",
    "Dinoyo": "R119 Dinoyo",
    "Tajur (R-121)": "R121 Tajur",
    "Cikupa": "R124 Cikupa",
    "Jatinegara": "R125 Jatinegara",
    "Timika (R-138)": "R138 Timika",
}


STEP VALIDASI

In [ ]:
df['Center_NORM'] = df['Center'].str.upper().str.strip()

unmapped = set(df['Center_NORM']) - set(k.upper() for k in DICT_TOKO_NTA.keys())
print("UNMAPPED CENTER:")
unmapped


Standarisasi Center

In [ ]:
df['Center_RAW'] = df['Center']               # simpan original
df['Center'] = (
    df['Center']
    .astype(str)
    .str.strip()
)


Flag: IN SCOPE vs FILTERED OUT

In [ ]:
df['IN_SCOPE_TOKO'] = df['Center'].isin(DICT_TOKO_NTA.keys())


Mapping hanya untuk yang IN SCOPE

In [ ]:
df['Center_MAPPED'] = df['Center'].map(DICT_TOKO_NTA)


VALIDASI JUMLAH TOKO

Total toko unik (raw)

In [ ]:
total_toko = df['Center'].nunique()
print("Total toko unik (raw):", total_toko)



Toko yang dipakai (IN SCOPE)

In [ ]:
toko_in_scope = df[df['IN_SCOPE_TOKO']]['Center'].nunique()
print("Toko IN SCOPE:", toko_in_scope)


In [ ]:
centers_in_scope = (
    df_filtered_in['Center']
    .dropna()
    .sort_values()
    .unique()
)

centers_in_scope


LIST TOKO YANG DI-FILTER OUT (UNTUK LAPORAN)

In [ ]:
df_filtered_out = (
    df[~df['IN_SCOPE_TOKO']]
    [['Center']]
    .drop_duplicates()
    .sort_values('Center')
)

df_filtered_out


VALIDASI DAMPAK KE ANGKA

Jumlah Baris

In [ ]:
print("Total baris data:", len(df))
print("Baris IN SCOPE:", df['IN_SCOPE_TOKO'].sum())
print("Baris FILTERED OUT:", (~df['IN_SCOPE_TOKO']).sum())


Dampak ke metrik utama

In [ ]:
summary = df.groupby('IN_SCOPE_TOKO')[[
    'Jumlah Diaktifkan',
    'Kredit yg Digunakan'
]].sum()

summary


CEK STRUKTUR DASAR MAPPING MESIN

In [ ]:
df[['Game Title', 'GT_FINAL']].sample(10)


CEK APAKAH MASIH ADA VARIASI KE 1 GT_FINAL

In [ ]:
audit_variasi = (
    df.groupby('GT_FINAL')['Game Title']
    .nunique()
    .sort_values(ascending=False)
    .reset_index(name='jumlah_variasi')
)

audit_variasi.head(20)
audit_variasi[audit_variasi['jumlah_variasi'] > 1]

In [ ]:
audit_detail = (
    df.groupby('GT_FINAL')['Game Title']
    .unique()
    .reset_index()
)

audit_detail[audit_detail['Game Title'].apply(len) > 1]


CEK GT_FINAL MIRIP (TYPO / TYPO HALUS)

In [ ]:
from difflib import get_close_matches

gt_list = sorted(df['GT_FINAL'].unique())

suspect_pairs = []

for gt in gt_list:
    matches = get_close_matches(gt, gt_list, n=3, cutoff=0.85)
    for m in matches:
        if gt != m:
            suspect_pairs.append((gt, m))

suspect_pairs[:20]


Merge Map

In [ ]:
GT_MERGE_MAP = {
    # --- AIR HOCKEY ---
    'AIR HOCKEY JR': 'AIR HOCKEY',

    # --- ALIENS ARMAGEDDON ---
    'ALIEN ARMAGEDON': 'ALLIENS ARMAGEDDON',

    # --- ANIMAL KAISER ---
    'ANIMAL KAISAR': 'ANIMAL KAISER',

    # --- APPLE FRENZY ---
    'APLE FRENZY': 'APPLE FRENZY',
    'APPLE FRENZY 1': 'APPLE FRENZY',
    'APPLE FRENZY 2': 'APPLE FRENZY',

    # --- APOLLO ---
    'APOLO': 'APPOLO',

    # --- AROUND THE WORLD ---
    'AROUND WORLD': 'AROUND THE WORLD',

    # --- BABY BALLOON HOCKEY ---
    'BABY BALLON HOCKEY': 'BABY BALOON HOCKEY'
}


CEK CEPAT SETELAH APPLY

In [ ]:
# Apply
df['GT_FINAL'] = df['GT_FINAL'].replace(GT_MERGE_MAP)

# Assertion
assert not (
    set(GT_MERGE_MAP.keys()) & set(df['GT_FINAL'].unique())
), "Masih ada GT lama yang belum ter-merge"


# EDA

Cek Struktur & Tipe Data

In [ ]:
df[['Jumlah Diaktifkan', 'Kredit yg Digunakan']].dtypes


Statistik Ringkas

In [ ]:
df[['Jumlah Diaktifkan', 'Kredit yg Digunakan']].describe()


Top Value Check

In [ ]:
df.sort_values('Kredit yg Digunakan', ascending=False).head(10)[
    ['Center', 'GT_FINAL', 'Jumlah Diaktifkan', 'Kredit yg Digunakan']
]


Filter out non mesin

In [ ]:
EXCLUDE_GT = [
    'KIDDY', 'KIDDIE', 'KIDDIELAND',
    'TICKET', 'ZONE', 'LAND' , "SALDO", "SOFT"
]

mask_exclude = df['GT_FINAL'].str.contains(
    '|'.join(EXCLUDE_GT),
    case=False,
    na=False
)

print("Row non-mesin:", mask_exclude.sum())

df = df[~mask_exclude].copy()


Top Value Check

In [ ]:
df.sort_values('Kredit yg Digunakan', ascending=False).head(10)[
    ['Center', 'GT_FINAL', 'Jumlah Diaktifkan', 'Kredit yg Digunakan']
]

Omset mesin 2024 vs 2025

In [ ]:
df_compare = (
    df
    .groupby('Tahun')['Kredit yg Digunakan']
    .sum()
    .reset_index()
)

df_compare


In [ ]:
df.describe()

Filter out Kiddie land dll yang bukan mesin

In [ ]:
df['Kategori Game'].value_counts()


Hapus KIDDIE LAND dan E-TICKET

In [ ]:
exclude_categories = ['KIDDIE LAND', 'E-TICKET']

print("Jumlah data sebelum:", len(df))

df = df[~df['Kategori Game'].isin(exclude_categories)]

print("Jumlah data sesudah :", len(df))


In [ ]:
df['Kategori Game'].value_counts()


In [ ]:
df_compare = (
    df
    .groupby('Tahun')['Kredit yg Digunakan']
    .sum()
    .reset_index()
)

df_compare

In [ ]:
df.sort_values('Kredit yg Digunakan', ascending=False).head(10)[
    ['Center', 'GT_FINAL', 'Jumlah Diaktifkan', 'Kategori Game', 'Kredit yg Digunakan', 'Bulan', 'Tahun']
]

Barplot komparasi

In [ ]:
#jadikan format rupiah
def format_id(x):
    return f"{int(x):,}".replace(",", ".")
#pastikan datetime
df['Tahun'] = df['Tanggal'].dt.year
df['Bulan'] = df['Tanggal'].dt.month

yearly_compare = (
    df[df['Tahun'].isin([2024, 2025])]
    .groupby('Tahun')['Kredit yg Digunakan']
    .sum()
)

fig, ax = plt.subplots(figsize=(6,4))

bars = ax.bar(
    yearly_compare.index.astype(str),
    yearly_compare.values
)

ax.set_title("Perbandingan Total Kredit Mesin\n2024 vs 2025")
ax.set_ylabel("Total Kredit Digunakan")
ax.set_xlabel("Tahun")

# Tambahkan label angka di atas bar
for bar in bars:
    height = bar.get_height()
    ax.text(
        bar.get_x() + bar.get_width()/2,
        height,
        format_id(height),
        ha='center',
        va='bottom',
        fontsize=9
    )

plt.tight_layout()
plt.show()


LINE CHART – Tren Bulanan

In [ ]:
monthly_trend = (
    df[df['Tahun'].isin([2024, 2025])]
    .groupby(['Tahun', 'Bulan'])['Kredit yg Digunakan']
    .sum()
    .reset_index()
)

plt.figure(figsize=(10,4))

for tahun in [2024, 2025]:
    data = monthly_trend[monthly_trend['Tahun'] == tahun]
    
    plt.plot(
        data['Bulan'],
        data['Kredit yg Digunakan'],
        marker='o',
        label=str(tahun)
    )
    
    # Tambahkan label di titik
    for _, row in data.iterrows():
        plt.text(
            row['Bulan'],
            row['Kredit yg Digunakan'],
            format_id(row['Kredit yg Digunakan']),
            fontsize=8,
            ha='center',
            va='bottom'
        )

plt.title("Tren Bulanan Kredit Mesin\n2024 vs 2025")
plt.xlabel("Bulan")
plt.ylabel("Total Kredit Digunakan")
plt.xticks(range(1,13))
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()


Plot tren toko aktif

In [ ]:

#jadikan format rupiah
def format_id(x):
    return f"{int(x):,}".replace(",", ".")
#pastikan datetime
df_filtered_in['Tahun'] = df_filtered_in['Tanggal'].dt.year
df_filtered_in['Bulan'] = df_filtered_in['Tanggal'].dt.month

yearly_compare = (
    df_filtered_in[df_filtered_in['Tahun'].isin([2024, 2025])]
    .groupby('Tahun')['Kredit yg Digunakan']
    .sum()
)

fig, ax = plt.subplots(figsize=(6,4))

bars = ax.bar(
    yearly_compare.index.astype(str),
    yearly_compare.values
)

ax.set_title("Perbandingan Total Kredit Mesin\n2024 vs 2025")
ax.set_ylabel("Total Kredit Digunakan")
ax.set_xlabel("Tahun")

# Tambahkan label angka di atas bar
for bar in bars:
    height = bar.get_height()
    ax.text(
        bar.get_x() + bar.get_width()/2,
        height,
        format_id(height),
        ha='center',
        va='bottom',
        fontsize=9
    )

plt.tight_layout()
plt.show()


Export df

In [ ]:
'''
df.to_excel("df_full.xlsx", index=False)
'''